In [16]:
import eons
import neuro
import risp
import speech2spikes
import os
import torchaudio
import torch

PATH_SOUNDS = '/home/joao/dev/MLAudio/shotspotter/data/gunshots'

# Look into possible enabling cumsum (see github)
s2s = speech2spikes.S2S()

# workaround to configure to 12khz sample rate (s2s.configure() didnt work)
s2s._default_spec_kwargs = {
    "sample_rate": 12000,
    "n_mels": 20,
    "n_fft": 512,
    "f_min": 20,
    "f_max": 4000,
    "hop_length": 80,
}
s2s.transform = torchaudio.transforms.MelSpectrogram(**s2s._default_spec_kwargs)

# Some constants
NUM_INPUT_NEURONS = 80 # see paper
NUM_OUTPUT_NEURONS = 2
NUM_SYNAPSES = 500
NUM_HIDDEN_NEURONS = 100
POP_SIZE = 50

MOA = neuro.MOA()
MOA.seed(23456789, '')

In [17]:
file_paths = [PATH_SOUNDS+'/'+fn for fn in os.listdir(PATH_SOUNDS)]

In [18]:
# we want to convert our sound file into spikes
def to_spikes(path):
    data, rate = torchaudio.load(path)

    trains, targets = s2s([(data, torch.tensor(0))])
    print(trains.shape)

    trains = trains[0]
    
    # LOOK INTO EXACTLY WHAT CUMSUM IS DOING so we can figure out conversion to spikes
    # with cumsum=False (default) looks like it makes sense
    # I think we dont want cumsum (not entirely sure of what its doing)

    print(trains[0][0])

    spikes = []
    spike_id_count = 0
    for i in range(2):
        for channel in trains[i]:
            pos_channel, neg_channel = [], []
            pos_id, neg_id = spike_id_count, spike_id_count+1
            for t in range(len(channel)):
                if channel[t] == 1:
                    pos_channel.append(neuro.Spike(pos_id, t, 1))
                    neg_channel.append(neuro.Spike(neg_id, t, 0))
                elif channel[t] == -1:
                    neg_channel.append(neuro.Spike(neg_id, t, 1))
                    pos_channel.append(neuro.Spike(pos_id, t, 0))
                else:
                    pos_channel.append(neuro.Spike(pos_id, t, 0))
                    neg_channel.append(neuro.Spike(neg_id, t, 0))

            spikes.append(pos_channel.copy())
            spikes.append(neg_channel.copy())

            spike_id_count += 2

    return spikes


# debug output of to_spikes
def print_spikes(spks, channel_id):
    # channel id from 0-79
    for i in range(len(spks[1])):
        print(f'{spks[channel_id][i].value:.0f}', end=', ')
            
spikes = to_spikes(file_paths[0])
print_spikes(spikes, 0)

torch.Size([1, 2, 20, 301])
tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,
         0., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  0., -1.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.,  0., -1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0., -1., -1.,  0.,
         1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         1.,  0.,  0.,  0.,  1.,  0.

In [23]:
# Configure RISP and EONS
risp_config = {
  "min_weight": -1,
  "max_weight": 1,
  "min_threshold": -1,
  "max_threshold": 1,
  "min_potential": -1,
  "max_delay": 10,
  "discrete": False
}

eons_param = {
    "starting_nodes": NUM_HIDDEN_NEURONS,
    "starting_edges": NUM_SYNAPSES,
    "merge_rate": 0,
    "population_size": POP_SIZE,
    "multi_edges": 0,
    "crossover_rate": 0.9,
    "mutation_rate": 0.9,
    "selection_type": "tournament",
    "tournament_size_factor": 0.1,
    "tournament_best_net_factor": 0.9,
    "random_factor": 0.10,
    "num_mutations": 10,
    "node_mutations": { "Threshold": 1.0 },
    "net_mutations": { },
    "edge_mutations": { "Weight": 0.65 , "Delay": 0.35,  },
    "num_best" : 4,
    "add_node_rate": 0.75,
    "delete_node_rate": 0.25,
    "add_edge_rate": 0.75,
    "delete_edge_rate": 0.25,
    "node_params_rate": 2.5,
    "edge_params_rate": 2.5,
    "net_params_rate" : 0
}

proc = risp.Processor(risp_config)
eons_inst = eons.EONS(eons_param)

In [20]:
# set up template network  (inputs and outputs) for eons
template_net = neuro.Network()
template_net.set_properties(proc.get_network_properties())

for i in range(NUM_INPUT_NEURONS):
    node = template_net.add_node(i)
    node.set("Threshold", 1)
    template_net.add_input(i)

for i in range(NUM_INPUT_NEURONS, NUM_INPUT_NEURONS+NUM_OUTPUT_NEURONS):
    node = template_net.add_node(i)
    node.set("Threshold", 1)
    template_net.add_output(i)

In [29]:
# Debugging or other utilities 
def network_details(nw, log_json=False):
    net_json = nw.as_json()

    if log_json:
        print(net_json)
    
    print(f'Network has {len(net_json["Edges"])} synapses and {len(net_json["Nodes"])} neurons')

    # check if all out nodes have an incoming synapse
    out_ids = net_json['Outputs']
    for edge in net_json['Edges']:
        if edge['to'] in out_ids:
            out_ids.remove(edge['to'])

    if len(out_ids) == 0:
        print('All outputs have incoming connections')
    else:
        print(f'Outputs {out_ids} have no incoming connections')

def compute_fitness(net):
    return 0

In [37]:
# Training
EPOCH_COUNT = 100
eons_inst.set_template_network(template_net)
pop = eons_inst.generate_population(eons_param, MOA.random_integer())

for i in range(EPOCH_COUNT):
    print(f'We are in epoch {i}')
    fits = [compute_fitness(n.network) for n in pop.networks]

    pop = eons_inst.do_epoch(pop, fits, eons_param)


network_details(pop.networks[0].network)

We are in epoch 0


NameError: name 'compute_fitness' is not defined